In [18]:
from huggingface_hub import hf_hub_download
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoformerForPrediction, AutoformerConfig
import torch.optim as optim
import sys
import torch
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
from nbeats_pytorch.model import NBeatsNet
from torch.utils.data import DataLoader, TensorDataset

sys.path.append('/home/noam.koren/multiTS/NFT/')

from models.training_functions import get_data
from dicts import data_to_num_vars_dict

In [19]:
data='air_quality' 
lookback=30
horizon=1
num_vars=data_to_num_vars_dict[data]
series=None
print_stats=False

In [20]:
train_X, train_y, val_X, val_y, test_X, test_y = get_data(
        data=data, 
        lookback=lookback, 
        horizon=horizon,
        n_series=num_vars,
        series=series,
        print_stats=print_stats
        )
        
train_X = train_X.reshape(-1, lookback, num_vars)
train_y = train_y.reshape(-1, horizon, num_vars)

val_X = val_X.reshape(-1, lookback, num_vars)
val_y = val_y.reshape(-1, horizon, num_vars)

test_X = test_X.reshape(-1, lookback, num_vars)
test_y = test_y.reshape(-1, horizon, num_vars)

# Assuming X and y are your dataset loaded into PyTorch Datasets
class ForecastingDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Prepare your data (X, y) before this
dataset = ForecastingDataset(train_X, train_y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [21]:
# Model and optimizer
configuration = AutoformerConfig(
    prediction_length=horizon
)
model = AutoformerForPrediction(configuration)
optimizer = optim.Adam(model.parameters(), lr=1e-5)


In [23]:

# Reshape data according to the specifications
train_X = train_X.reshape(-1, lookback, num_vars)
train_y = train_y.reshape(-1, horizon, num_vars)
val_X = val_X.reshape(-1, lookback, num_vars)
val_y = val_y.reshape(-1, horizon, num_vars)
test_X = test_X.reshape(-1, lookback, num_vars)
test_y = test_y.reshape(-1, horizon, num_vars)

# Assuming X and y are your dataset loaded into PyTorch Datasets
class ForecastingDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        # Assuming all past values are observed
        self.past_observed_mask = torch.ones_like(X, dtype=torch.bool)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.past_observed_mask[idx]

# Prepare your data
dataset = ForecastingDataset(train_X, train_y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Model and optimizer configuration
configuration = AutoformerConfig(prediction_length=horizon)
model = AutoformerForPrediction(configuration)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Training loop
model.train()
num_epochs = 3  # Example: Set the number of epochs
for epoch in range(num_epochs):
    for batch_idx, (batch_X, batch_y, batch_mask) in enumerate(dataloader):
        optimizer.zero_grad()

        outputs = model(
            past_values=batch_X,
            future_values=batch_y,
            past_observed_mask=batch_mask,
            past_time_features=batch_X,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}")


TypeError: expected Tensor as element 1 in argument 0, but got NoneType